## 按区域检索与下载影像

用户可使用平台内置或自主上传的矢量文件，进行数据的检索（以 *Sentine-2 L2A* 为例），再进行数据筛选、拼接、裁剪等操作后，将数据导出至 *我的数据* 中。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### 定义矢量区域
使用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=50659c#heading-0" target="_blank">FeatureCollection</a>* 引用平台内置或自主上传的矢量边界，定义检索数据的区域。利用 *<a href="https://pre-engine-aiearth.aliyun.com/docs/page/api?d=b3f315" target="_blank">aie.Map</a>* 构造一个地图组件 *Map* 对象，通过 *<a href="https://pre-engine-aiearth.aliyun.com/docs/page/api?d=834403" target="_blank">aie.Map.addLayer</a>* 用于地图可视化渲染不同图层。


In [2]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省')) 
            
map = aie.Map(
    center=region.getCenter(),
    height=800,
    zoom=6
)

vis_params = {
    'color': '#00FF00'
}

map.addLayer(
    region,
    vis_params,
    'region',
    bounds=region.getBounds()
)
map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

### *Sentinel-2* 数据检索
定义函数 *s2_collection* ，实现按区域、时间、云量等条件的 *Sentinel-2* 数据检索，返回哨兵单景 *s2 image* 和进行镶嵌、裁剪后的 *s2 mosaic image* 。

In [3]:
def s2_collection(start_date, end_date):
    s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
            .filterBounds(region) \
            .filterDate(start_date, end_date) \
            .filter('eo:cloud_cover<20')
    mosaic_image = s2.median().clipToCollection(region)
    return s2, mosaic_image
    
s2, s2_mosaic = s2_collection('2021-04-01', '2022-08-30')

### 数据可视化
对 *S2* 数据进行波段组合可视化，常用波段：真彩色 *['B4', 'B3', 'B2']* 、假彩色 *['B8', 'B4', 'B3']* 、假彩色 *['B12', 'B11', 'B4']* 等。

In [4]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],   
    'min': 0,
    'max': 3500
}
    
map.addLayer(
    s2_mosaic,
    vis_params,
    'Ture color',
    bounds=region.getBounds()
)
map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

### 影像导出
使用 *<a href="https://pre-engine-aiearth.aliyun.com/docs/page/api?d=337f2c" target="_blank">Export.image.toAsset</a>* 将数据导出至我的数据中，可以通过 *scale* 参数指定导出的分辨率（ *单位：米* ）。

In [7]:
# 导出镶嵌影像
task = aie.Export.image.toAsset(s2_mosaic, 's2_mosaic', 200)
task.start()

